In [4]:
import hydra
import pickle
from collections import defaultdict
from pathlib import Path

import librosa
import numpy as np
import omegaconf
import textgrids
import torch
from tqdm import tqdm
from transformers import AutoModel

with hydra.initialize(version_base=None, config_path="../../../conf"):
    cfg = hydra.compose(config_name="config")

/home/minami/lip2sp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/minami/lip2sp/.venv/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [18]:
audio_path = '/home/minami/dataset/lip/wav/F02_kablab/ATR503_i44.wav'
wav, _ = librosa.load(str(audio_path), sr=cfg.data.audio.sr)
wav, _ = librosa.load(str(audio_path), sr=cfg.data.audio.sr)
wav = wav[: wav.shape[0] - (wav.shape[0] % (cfg.data.audio.sr // 100))]
wav = np.pad(
    wav,
    (0, int(cfg.data.audio.sr // 100 * 2)),
    mode="constant",
    constant_values=0,
)
wav_input = torch.from_numpy(wav).unsqueeze(0)

hubert = AutoModel.from_pretrained(cfg.model.decoder.hubert.model_name).cuda()
feature_extractor_output = hubert.feature_extractor(wav_input.cuda())
feature_extractor_output = feature_extractor_output.permute(
    0, 2, 1
)  # (B, T, C)
feature_prj_output = hubert.feature_projection(
    feature_extractor_output
)  # (B, T, C)
feature_prj_output = hubert.encoder.pos_conv_embed(feature_prj_output)
feature_prj_output = hubert.encoder.layer_norm(feature_prj_output)
feature_prj_output = hubert.encoder.dropout(feature_prj_output)

transformer_layer_outputs = []
transformer_layer_output = feature_prj_output
for layer in hubert.encoder.layers:
    transformer_layer_output = layer(transformer_layer_output)
    # transformer_layer_outputs.append(transformer_layer_output)

AttributeError: 'tuple' object has no attribute 'size'